In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.utils import shuffle
from sklearn.preprocessing import StandardScaler
from sklearn import svm
from sklearn.metrics import classification_report, accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.ensemble import RandomForestClassifier
import time
import collections

In [2]:
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline as ImbPipeline
from sklearn.svm import SVC

In [3]:
X_train= pd.read_pickle('X_train.pkl')
y_train= pd.read_pickle('y_train.pkl')
X_test= pd.read_pickle('X_test.pkl')
y_test= pd.read_pickle('y_test.pkl')

### PIPELINE: RF-> SMOTE -> CV

### 1- scoring=accuracy

In [5]:
# Create a pipeline that first applies SMOTE and then scales the data
pipeline = ImbPipeline([
    ('smote', SMOTE(random_state=345)),
    ('scaler', StandardScaler()),
    ('rf', RandomForestClassifier(random_state=345))  # Replace 'svm' with 'rf' for Random Forest
])

# Define the parameter grid for the Random Forest classifier
# Note: Specify 'rf__' before each parameter name to indicate these parameters are for the Random Forest step in the pipeline
param_grid = {
    'smote__k_neighbors': [2, 3, 4],  # Example range; adjust based on your dataset characteristics
    'rf__n_estimators': [10, 50, 100, 200],  # Number of trees in the forest
    'rf__max_depth': [None, 10, 20, 30],  # Maximum depth of the trees
    'rf__min_samples_split': [2, 5, 10],  # Minimum number of samples required to split an internal node
    'rf__min_samples_leaf': [1, 2, 4]  # Minimum number of samples required to be at a leaf node
}

# Create a grid search object with cross-validation
grid_search = GridSearchCV(pipeline, param_grid, refit=True, cv=5, scoring='accuracy', verbose=10,n_jobs=-1)


In [6]:
start = time.time()
# Fit the grid search object to the training data (NOT the SMOTE-resampled data)
grid_search.fit(X_train, y_train)  # Use your original training data here
end = time.time()

dt = end - start
print('delta t =', dt)

Fitting 5 folds for each of 432 candidates, totalling 2160 fits
[CV 1/5; 1/432] START rf__max_depth=None, rf__min_samples_leaf=1, rf__min_samples_split=2, rf__n_estimators=10, smote__k_neighbors=2
[CV 1/5; 1/432] END rf__max_depth=None, rf__min_samples_leaf=1, rf__min_samples_split=2, rf__n_estimators=10, smote__k_neighbors=2;, score=0.989 total time=   0.5s
[CV 2/5; 1/432] START rf__max_depth=None, rf__min_samples_leaf=1, rf__min_samples_split=2, rf__n_estimators=10, smote__k_neighbors=2
[CV 2/5; 1/432] END rf__max_depth=None, rf__min_samples_leaf=1, rf__min_samples_split=2, rf__n_estimators=10, smote__k_neighbors=2;, score=0.989 total time=   0.4s
[CV 3/5; 1/432] START rf__max_depth=None, rf__min_samples_leaf=1, rf__min_samples_split=2, rf__n_estimators=10, smote__k_neighbors=2
[CV 3/5; 1/432] END rf__max_depth=None, rf__min_samples_leaf=1, rf__min_samples_split=2, rf__n_estimators=10, smote__k_neighbors=2;, score=0.993 total time=   0.4s
[CV 4/5; 1/432] START rf__max_depth=None, rf_

In [7]:
# Print results
results = grid_search.cv_results_
for mean_score, std_score, params in zip(results['mean_test_score'], results['std_test_score'], results['params']):
    print(f"Mean test score: {mean_score:.3f} (+/-{std_score:.3f}) for {params}")

Mean test score: 0.990 (+/-0.001) for {'rf__max_depth': None, 'rf__min_samples_leaf': 1, 'rf__min_samples_split': 2, 'rf__n_estimators': 10, 'smote__k_neighbors': 2}
Mean test score: 0.992 (+/-0.003) for {'rf__max_depth': None, 'rf__min_samples_leaf': 1, 'rf__min_samples_split': 2, 'rf__n_estimators': 10, 'smote__k_neighbors': 3}
Mean test score: 0.992 (+/-0.001) for {'rf__max_depth': None, 'rf__min_samples_leaf': 1, 'rf__min_samples_split': 2, 'rf__n_estimators': 10, 'smote__k_neighbors': 4}
Mean test score: 0.990 (+/-0.001) for {'rf__max_depth': None, 'rf__min_samples_leaf': 1, 'rf__min_samples_split': 2, 'rf__n_estimators': 50, 'smote__k_neighbors': 2}
Mean test score: 0.991 (+/-0.002) for {'rf__max_depth': None, 'rf__min_samples_leaf': 1, 'rf__min_samples_split': 2, 'rf__n_estimators': 50, 'smote__k_neighbors': 3}
Mean test score: 0.991 (+/-0.002) for {'rf__max_depth': None, 'rf__min_samples_leaf': 1, 'rf__min_samples_split': 2, 'rf__n_estimators': 50, 'smote__k_neighbors': 4}
Mean

In [8]:
# Print the best parameters and score
print('Best parameters: ', grid_search.best_params_)
print('Best score: ', grid_search.best_score_)
# Access the best model
best_model = grid_search.best_estimator_

Best parameters:  {'rf__max_depth': None, 'rf__min_samples_leaf': 1, 'rf__min_samples_split': 10, 'rf__n_estimators': 10, 'smote__k_neighbors': 4}
Best score:  0.9936119734484239


In [9]:
best_model

Pipeline(steps=[('smote', SMOTE(k_neighbors=4, random_state=345)),
                ('scaler', StandardScaler()),
                ('rf',
                 RandomForestClassifier(min_samples_split=10, n_estimators=10,
                                        random_state=345))])

In [10]:
y_pred = grid_search.best_estimator_.predict(X_test)

# Calculate the accuracy on the test set
test_accuracy = accuracy_score(y_test, y_pred)

print(f'Test Set Accuracy: {test_accuracy * 100:.2f}%')

Test Set Accuracy: 99.72%


In [11]:
# Print the classification report
y_pred = grid_search.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

          -1       1.00      1.00      1.00       350
           1       1.00      0.67      0.80         3

    accuracy                           1.00       353
   macro avg       1.00      0.83      0.90       353
weighted avg       1.00      1.00      1.00       353



In [12]:
cm = confusion_matrix(y_test, y_pred)

# Print the confusion matrix
print("Confusion Matrix:")
print(cm)

Confusion Matrix:
[[350   0]
 [  1   2]]


### 2-scoring= f1

In [28]:
# Create a pipeline that first applies SMOTE and then scales the data
pipeline = ImbPipeline([
    ('smote', SMOTE(random_state=345)),
    ('scaler', StandardScaler()),
    ('rf', RandomForestClassifier(random_state=345))  # Replace 'svm' with 'rf' for Random Forest
])

# Define the parameter grid for the Random Forest classifier
# Note: Specify 'rf__' before each parameter name to indicate these parameters are for the Random Forest step in the pipeline
param_grid = {
    'smote__k_neighbors': [2, 3, 4],  # Example range; adjust based on your dataset characteristics
    'rf__n_estimators': [10, 50, 100, 200],  # Number of trees in the forest
    'rf__max_depth': [None, 10, 20, 30],  # Maximum depth of the trees
    'rf__min_samples_split': [2, 5, 10],  # Minimum number of samples required to split an internal node
    'rf__min_samples_leaf': [1, 2, 4]  # Minimum number of samples required to be at a leaf node
}

# Create a grid search object with cross-validation
grid_search = GridSearchCV(pipeline, param_grid, refit=True, cv=5, scoring='f1', verbose=10,n_jobs=-1)


In [29]:
start = time.time()
# Fit the grid search object to the training data (NOT the SMOTE-resampled data)
grid_search.fit(X_train, y_train)  # Use your original training data here
end = time.time()

dt = end - start
print('delta t =', dt)

Fitting 5 folds for each of 432 candidates, totalling 2160 fits
[CV 2/5; 1/432] START rf__max_depth=None, rf__min_samples_leaf=1, rf__min_samples_split=2, rf__n_estimators=10, smote__k_neighbors=2
[CV 1/5; 1/432] START rf__max_depth=None, rf__min_samples_leaf=1, rf__min_samples_split=2, rf__n_estimators=10, smote__k_neighbors=2
[CV 4/5; 1/432] START rf__max_depth=None, rf__min_samples_leaf=1, rf__min_samples_split=2, rf__n_estimators=10, smote__k_neighbors=2
[CV 3/5; 1/432] START rf__max_depth=None, rf__min_samples_leaf=1, rf__min_samples_split=2, rf__n_estimators=10, smote__k_neighbors=2
[CV 1/5; 2/432] START rf__max_depth=None, rf__min_samples_leaf=1, rf__min_samples_split=2, rf__n_estimators=10, smote__k_neighbors=3
[CV 2/5; 2/432] START rf__max_depth=None, rf__min_samples_leaf=1, rf__min_samples_split=2, rf__n_estimators=10, smote__k_neighbors=3
[CV 5/5; 2/432] START rf__max_depth=None, rf__min_samples_leaf=1, rf__min_samples_split=2, rf__n_estimators=10, smote__k_neighbors=3
[CV 5

In [30]:
# Print results
results = grid_search.cv_results_
for mean_score, std_score, params in zip(results['mean_test_score'], results['std_test_score'], results['params']):
    print(f"Mean test score: {mean_score:.3f} (+/-{std_score:.3f}) for {params}")

Mean test score: 0.080 (+/-0.160) for {'rf__max_depth': None, 'rf__min_samples_leaf': 1, 'rf__min_samples_split': 2, 'rf__n_estimators': 10, 'smote__k_neighbors': 2}
Mean test score: 0.160 (+/-0.320) for {'rf__max_depth': None, 'rf__min_samples_leaf': 1, 'rf__min_samples_split': 2, 'rf__n_estimators': 10, 'smote__k_neighbors': 3}
Mean test score: 0.100 (+/-0.200) for {'rf__max_depth': None, 'rf__min_samples_leaf': 1, 'rf__min_samples_split': 2, 'rf__n_estimators': 10, 'smote__k_neighbors': 4}
Mean test score: 0.080 (+/-0.160) for {'rf__max_depth': None, 'rf__min_samples_leaf': 1, 'rf__min_samples_split': 2, 'rf__n_estimators': 50, 'smote__k_neighbors': 2}
Mean test score: 0.080 (+/-0.160) for {'rf__max_depth': None, 'rf__min_samples_leaf': 1, 'rf__min_samples_split': 2, 'rf__n_estimators': 50, 'smote__k_neighbors': 3}
Mean test score: 0.080 (+/-0.160) for {'rf__max_depth': None, 'rf__min_samples_leaf': 1, 'rf__min_samples_split': 2, 'rf__n_estimators': 50, 'smote__k_neighbors': 4}
Mean

In [31]:
# Print the best parameters and score
print('Best parameters: ', grid_search.best_params_)
print('Best score: ', grid_search.best_score_)
# Access the best model
best_model = grid_search.best_estimator_

Best parameters:  {'rf__max_depth': None, 'rf__min_samples_leaf': 4, 'rf__min_samples_split': 10, 'rf__n_estimators': 10, 'smote__k_neighbors': 3}
Best score:  0.26666666666666666


In [32]:
best_model

Pipeline(steps=[('smote', SMOTE(k_neighbors=3, random_state=345)),
                ('scaler', StandardScaler()),
                ('rf',
                 RandomForestClassifier(min_samples_leaf=4,
                                        min_samples_split=10, n_estimators=10,
                                        random_state=345))])

In [33]:
y_pred = grid_search.best_estimator_.predict(X_test)

# Calculate the accuracy on the test set
test_accuracy = accuracy_score(y_test, y_pred)

print(f'Test Set Accuracy: {test_accuracy * 100:.2f}%')

Test Set Accuracy: 99.72%


In [34]:
# Print the classification report
y_pred = grid_search.predict(X_test)
print(classification_report(y_test, y_pred))

cm = confusion_matrix(y_test, y_pred)

# Print the confusion matrix
print("Confusion Matrix:")
print(cm)

              precision    recall  f1-score   support

          -1       1.00      1.00      1.00       350
           1       1.00      0.67      0.80         3

    accuracy                           1.00       353
   macro avg       1.00      0.83      0.90       353
weighted avg       1.00      1.00      1.00       353

Confusion Matrix:
[[350   0]
 [  1   2]]


### 3-scoring= recall

In [14]:
# Create a pipeline that first applies SMOTE and then scales the data
pipeline = ImbPipeline([
    ('smote', SMOTE(random_state=345)),
    ('scaler', StandardScaler()),
    ('rf', RandomForestClassifier(random_state=345))  # Replace 'svm' with 'rf' for Random Forest
])

# Define the parameter grid for the Random Forest classifier
# Note: Specify 'rf__' before each parameter name to indicate these parameters are for the Random Forest step in the pipeline
param_grid = {
    'smote__k_neighbors': [2, 3, 4],  # Example range; adjust based on your dataset characteristics
    'rf__n_estimators': [10, 50, 100, 200],  # Number of trees in the forest
    'rf__max_depth': [None, 10, 20, 30],  # Maximum depth of the trees
    'rf__min_samples_split': [2, 5, 10],  # Minimum number of samples required to split an internal node
    'rf__min_samples_leaf': [1, 2, 4]  # Minimum number of samples required to be at a leaf node
}

# Create a grid search object with cross-validation
grid_search = GridSearchCV(pipeline, param_grid, refit=True, cv=5, scoring='recall', verbose=10,n_jobs=-1)


In [15]:
start = time.time()
# Fit the grid search object to the training data (NOT the SMOTE-resampled data)
grid_search.fit(X_train, y_train)  # Use your original training data here
end = time.time()

dt = end - start
print('delta t =', dt)

Fitting 5 folds for each of 432 candidates, totalling 2160 fits
[CV 2/5; 1/432] START rf__max_depth=None, rf__min_samples_leaf=1, rf__min_samples_split=2, rf__n_estimators=10, smote__k_neighbors=2
[CV 3/5; 1/432] START rf__max_depth=None, rf__min_samples_leaf=1, rf__min_samples_split=2, rf__n_estimators=10, smote__k_neighbors=2
[CV 1/5; 1/432] START rf__max_depth=None, rf__min_samples_leaf=1, rf__min_samples_split=2, rf__n_estimators=10, smote__k_neighbors=2
[CV 5/5; 1/432] START rf__max_depth=None, rf__min_samples_leaf=1, rf__min_samples_split=2, rf__n_estimators=10, smote__k_neighbors=2
[CV 1/5; 2/432] START rf__max_depth=None, rf__min_samples_leaf=1, rf__min_samples_split=2, rf__n_estimators=10, smote__k_neighbors=3
[CV 2/5; 2/432] START rf__max_depth=None, rf__min_samples_leaf=1, rf__min_samples_split=2, rf__n_estimators=10, smote__k_neighbors=3
[CV 3/5; 2/432] START rf__max_depth=None, rf__min_samples_leaf=1, rf__min_samples_split=2, rf__n_estimators=10, smote__k_neighbors=3
[CV 4

In [16]:
# Print results
results = grid_search.cv_results_
for mean_score, std_score, params in zip(results['mean_test_score'], results['std_test_score'], results['params']):
    print(f"Mean test score: {mean_score:.3f} (+/-{std_score:.3f}) for {params}")

Mean test score: 0.067 (+/-0.133) for {'rf__max_depth': None, 'rf__min_samples_leaf': 1, 'rf__min_samples_split': 2, 'rf__n_estimators': 10, 'smote__k_neighbors': 2}
Mean test score: 0.133 (+/-0.267) for {'rf__max_depth': None, 'rf__min_samples_leaf': 1, 'rf__min_samples_split': 2, 'rf__n_estimators': 10, 'smote__k_neighbors': 3}
Mean test score: 0.067 (+/-0.133) for {'rf__max_depth': None, 'rf__min_samples_leaf': 1, 'rf__min_samples_split': 2, 'rf__n_estimators': 10, 'smote__k_neighbors': 4}
Mean test score: 0.067 (+/-0.133) for {'rf__max_depth': None, 'rf__min_samples_leaf': 1, 'rf__min_samples_split': 2, 'rf__n_estimators': 50, 'smote__k_neighbors': 2}
Mean test score: 0.067 (+/-0.133) for {'rf__max_depth': None, 'rf__min_samples_leaf': 1, 'rf__min_samples_split': 2, 'rf__n_estimators': 50, 'smote__k_neighbors': 3}
Mean test score: 0.067 (+/-0.133) for {'rf__max_depth': None, 'rf__min_samples_leaf': 1, 'rf__min_samples_split': 2, 'rf__n_estimators': 50, 'smote__k_neighbors': 4}
Mean

In [17]:
# Print the best parameters and score
print('Best parameters: ', grid_search.best_params_)
print('Best score: ', grid_search.best_score_)
# Access the best model
best_model = grid_search.best_estimator_

Best parameters:  {'rf__max_depth': None, 'rf__min_samples_leaf': 4, 'rf__min_samples_split': 10, 'rf__n_estimators': 10, 'smote__k_neighbors': 3}
Best score:  0.2333333333333333


In [18]:
best_model

Pipeline(steps=[('smote', SMOTE(k_neighbors=3, random_state=345)),
                ('scaler', StandardScaler()),
                ('rf',
                 RandomForestClassifier(min_samples_leaf=4,
                                        min_samples_split=10, n_estimators=10,
                                        random_state=345))])

In [19]:
y_pred = grid_search.best_estimator_.predict(X_test)

# Calculate the accuracy on the test set
test_accuracy = accuracy_score(y_test, y_pred)

print(f'Test Set Accuracy: {test_accuracy * 100:.2f}%')

Test Set Accuracy: 99.72%


In [20]:
# Print the classification report
y_pred = grid_search.predict(X_test)
print(classification_report(y_test, y_pred))


cm = confusion_matrix(y_test, y_pred)

# Print the confusion matrix
print("Confusion Matrix:")
print(cm)

              precision    recall  f1-score   support

          -1       1.00      1.00      1.00       350
           1       1.00      0.67      0.80         3

    accuracy                           1.00       353
   macro avg       1.00      0.83      0.90       353
weighted avg       1.00      1.00      1.00       353

Confusion Matrix:
[[350   0]
 [  1   2]]


### 4-scoring= precision

In [21]:
# Create a pipeline that first applies SMOTE and then scales the data
pipeline = ImbPipeline([
    ('smote', SMOTE(random_state=345)),
    ('scaler', StandardScaler()),
    ('rf', RandomForestClassifier(random_state=345))  # Replace 'svm' with 'rf' for Random Forest
])

# Define the parameter grid for the Random Forest classifier
# Note: Specify 'rf__' before each parameter name to indicate these parameters are for the Random Forest step in the pipeline
param_grid = {
    'smote__k_neighbors': [2, 3, 4],  # Example range; adjust based on your dataset characteristics
    'rf__n_estimators': [10, 50, 100, 200],  # Number of trees in the forest
    'rf__max_depth': [None, 10, 20, 30],  # Maximum depth of the trees
    'rf__min_samples_split': [2, 5, 10],  # Minimum number of samples required to split an internal node
    'rf__min_samples_leaf': [1, 2, 4]  # Minimum number of samples required to be at a leaf node
}

# Create a grid search object with cross-validation
grid_search = GridSearchCV(pipeline, param_grid, refit=True, cv=5, scoring='precision', verbose=10,n_jobs=-1)


In [22]:
start = time.time()
# Fit the grid search object to the training data (NOT the SMOTE-resampled data)
grid_search.fit(X_train, y_train)  # Use your original training data here
end = time.time()

dt = end - start
print('delta t =', dt)

Fitting 5 folds for each of 432 candidates, totalling 2160 fits
[CV 1/5; 1/432] START rf__max_depth=None, rf__min_samples_leaf=1, rf__min_samples_split=2, rf__n_estimators=10, smote__k_neighbors=2
[CV 4/5; 1/432] START rf__max_depth=None, rf__min_samples_leaf=1, rf__min_samples_split=2, rf__n_estimators=10, smote__k_neighbors=2
[CV 2/5; 1/432] START rf__max_depth=None, rf__min_samples_leaf=1, rf__min_samples_split=2, rf__n_estimators=10, smote__k_neighbors=2
[CV 5/5; 1/432] START rf__max_depth=None, rf__min_samples_leaf=1, rf__min_samples_split=2, rf__n_estimators=10, smote__k_neighbors=2
[CV 1/5; 2/432] START rf__max_depth=None, rf__min_samples_leaf=1, rf__min_samples_split=2, rf__n_estimators=10, smote__k_neighbors=3
[CV 3/5; 2/432] START rf__max_depth=None, rf__min_samples_leaf=1, rf__min_samples_split=2, rf__n_estimators=10, smote__k_neighbors=3
[CV 2/5; 2/432] START rf__max_depth=None, rf__min_samples_leaf=1, rf__min_samples_split=2, rf__n_estimators=10, smote__k_neighbors=3
[CV 4

/home/arb399/.conda/envs/ml_env/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


[CV 3/5; 1/432] END rf__max_depth=None, rf__min_samples_leaf=1, rf__min_samples_split=2, rf__n_estimators=10, smote__k_neighbors=2;, score=0.000 total time=   1.1s
[CV 4/5; 5/432] START rf__max_depth=None, rf__min_samples_leaf=1, rf__min_samples_split=2, rf__n_estimators=50, smote__k_neighbors=3
[CV 1/5; 2/432] END rf__max_depth=None, rf__min_samples_leaf=1, rf__min_samples_split=2, rf__n_estimators=10, smote__k_neighbors=3;, score=0.000 total time=   1.4s
[CV 2/5; 1/432] END rf__max_depth=None, rf__min_samples_leaf=1, rf__min_samples_split=2, rf__n_estimators=10, smote__k_neighbors=2;, score=0.000 total time=   1.5s
[CV 5/5; 5/432] START rf__max_depth=None, rf__min_samples_leaf=1, rf__min_samples_split=2, rf__n_estimators=50, smote__k_neighbors=3
[CV 3/5; 2/432] END rf__max_depth=None, rf__min_samples_leaf=1, rf__min_samples_split=2, rf__n_estimators=10, smote__k_neighbors=3;, score=0.000 total time=   1.5s
[CV 5/5; 1/432] END rf__max_depth=None, rf__min_samples_leaf=1, rf__min_sample

/home/arb399/.conda/envs/ml_env/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


[CV 1/5; 6/432] START rf__max_depth=None, rf__min_samples_leaf=1, rf__min_samples_split=2, rf__n_estimators=50, smote__k_neighbors=4
[CV 4/5; 2/432] END rf__max_depth=None, rf__min_samples_leaf=1, rf__min_samples_split=2, rf__n_estimators=10, smote__k_neighbors=3;, score=1.000 total time=   1.6s
[CV 2/5; 6/432] START rf__max_depth=None, rf__min_samples_leaf=1, rf__min_samples_split=2, rf__n_estimators=50, smote__k_neighbors=4
[CV 2/5; 2/432] END rf__max_depth=None, rf__min_samples_leaf=1, rf__min_samples_split=2, rf__n_estimators=10, smote__k_neighbors=3;, score=0.000 total time=   1.7s
[CV 3/5; 6/432] START rf__max_depth=None, rf__min_samples_leaf=1, rf__min_samples_split=2, rf__n_estimators=50, smote__k_neighbors=4
[CV 4/5; 6/432] START rf__max_depth=None, rf__min_samples_leaf=1, rf__min_samples_split=2, rf__n_estimators=50, smote__k_neighbors=4
[CV 5/5; 6/432] START rf__max_depth=None, rf__min_samples_leaf=1, rf__min_samples_split=2, rf__n_estimators=50, smote__k_neighbors=4
[CV 1/5

...

[CV 5/5; 432/432] END rf__max_depth=30, rf__min_samples_leaf=4, rf__min_samples_split=10, rf__n_estimators=200, smote__k_neighbors=4;, score=0.000 total time=   3.1s


/home/arb399/.conda/envs/ml_env/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


[CV 3/5; 431/432] END rf__max_depth=30, rf__min_samples_leaf=4, rf__min_samples_split=10, rf__n_estimators=200, smote__k_neighbors=3;, score=0.000 total time=   4.2s
[CV 4/5; 431/432] END rf__max_depth=30, rf__min_samples_leaf=4, rf__min_samples_split=10, rf__n_estimators=200, smote__k_neighbors=3;, score=0.250 total time=   4.1s
[CV 2/5; 432/432] END rf__max_depth=30, rf__min_samples_leaf=4, rf__min_samples_split=10, rf__n_estimators=200, smote__k_neighbors=4;, score=0.000 total time=   4.2s
[CV 4/5; 432/432] END rf__max_depth=30, rf__min_samples_leaf=4, rf__min_samples_split=10, rf__n_estimators=200, smote__k_neighbors=4;, score=0.333 total time=   4.1s


/home/arb399/.conda/envs/ml_env/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


[CV 3/5; 432/432] END rf__max_depth=30, rf__min_samples_leaf=4, rf__min_samples_split=10, rf__n_estimators=200, smote__k_neighbors=4;, score=0.000 total time=   4.4s
delta t = 231.2393000125885


In [23]:
# Print results
results = grid_search.cv_results_
for mean_score, std_score, params in zip(results['mean_test_score'], results['std_test_score'], results['params']):
    print(f"Mean test score: {mean_score:.3f} (+/-{std_score:.3f}) for {params}")

Mean test score: 0.100 (+/-0.200) for {'rf__max_depth': None, 'rf__min_samples_leaf': 1, 'rf__min_samples_split': 2, 'rf__n_estimators': 10, 'smote__k_neighbors': 2}
Mean test score: 0.200 (+/-0.400) for {'rf__max_depth': None, 'rf__min_samples_leaf': 1, 'rf__min_samples_split': 2, 'rf__n_estimators': 10, 'smote__k_neighbors': 3}
Mean test score: 0.200 (+/-0.400) for {'rf__max_depth': None, 'rf__min_samples_leaf': 1, 'rf__min_samples_split': 2, 'rf__n_estimators': 10, 'smote__k_neighbors': 4}
Mean test score: 0.100 (+/-0.200) for {'rf__max_depth': None, 'rf__min_samples_leaf': 1, 'rf__min_samples_split': 2, 'rf__n_estimators': 50, 'smote__k_neighbors': 2}
Mean test score: 0.100 (+/-0.200) for {'rf__max_depth': None, 'rf__min_samples_leaf': 1, 'rf__min_samples_split': 2, 'rf__n_estimators': 50, 'smote__k_neighbors': 3}
Mean test score: 0.100 (+/-0.200) for {'rf__max_depth': None, 'rf__min_samples_leaf': 1, 'rf__min_samples_split': 2, 'rf__n_estimators': 50, 'smote__k_neighbors': 4}
Mean

In [24]:
# Print the best parameters and score
print('Best parameters: ', grid_search.best_params_)
print('Best score: ', grid_search.best_score_)
# Access the best model
best_model = grid_search.best_estimator_

Best parameters:  {'rf__max_depth': None, 'rf__min_samples_leaf': 1, 'rf__min_samples_split': 10, 'rf__n_estimators': 10, 'smote__k_neighbors': 4}
Best score:  0.4


In [25]:
best_model

Pipeline(steps=[('smote', SMOTE(k_neighbors=4, random_state=345)),
                ('scaler', StandardScaler()),
                ('rf',
                 RandomForestClassifier(min_samples_split=10, n_estimators=10,
                                        random_state=345))])

In [26]:
y_pred = grid_search.best_estimator_.predict(X_test)

# Calculate the accuracy on the test set
test_accuracy = accuracy_score(y_test, y_pred)

print(f'Test Set Accuracy: {test_accuracy * 100:.2f}%')

Test Set Accuracy: 99.72%


In [27]:
# Print the classification report
y_pred = grid_search.predict(X_test)
print(classification_report(y_test, y_pred))


cm = confusion_matrix(y_test, y_pred)

# Print the confusion matrix
print("Confusion Matrix:")
print(cm)

              precision    recall  f1-score   support

          -1       1.00      1.00      1.00       350
           1       1.00      0.67      0.80         3

    accuracy                           1.00       353
   macro avg       1.00      0.83      0.90       353
weighted avg       1.00      1.00      1.00       353

Confusion Matrix:
[[350   0]
 [  1   2]]


In [35]:
type(cm)

numpy.ndarray